# Preparação de Dados (N1)

## 🗂️ [ Projeto ]

**Nome do Projeto:** [nome do projeto]  
**Dataset:** [Nome do dataset ou fonte de dados]  
**Objetivo Geral:** [Breve Explicação do propósito — ex.: “Analisar o comportamento de churn e identificar fatores de retenção de clientes.”]  

---

## 📋 Descrição Geral do Tratamento

Nesta etapa serão realizados os principais **processos de limpeza, transformação e padronização dos dados**.  
O foco é garantir que o dataset esteja **consistente, estruturado e pronto para análises posteriores**.

**Tarefas previstas:**  
- Leitura e diagnóstico inicial do dataset.  
- Tratamento de valores nulos, duplicados e outliers.  
- Conversão de tipos e padronização de nomes de colunas.  
- Codificação de variáveis categóricas e normalização de numéricas (quando necessário).  
- Geração de artefatos intermediários e relatório de qualidade dos dados.

## 🔧 Configuração do Projeto (Bootstrap)

Esta etapa realiza o **bootstrap do ambiente de execução**, preparando o notebook para funcionar em qualquer máquina — garantindo **reprodutibilidade**, **organização** e **autonomia**.  
Todas as dependências, caminhos e configurações são ajustadas automaticamente, sem necessidade de modificações manuais.

---

### ⚙️ Etapas realizadas

#### 🧭 1. Localização da raiz do projeto  
- Busca o arquivo `config/defaults.json` subindo diretórios até encontrá-lo.  
- Define `PROJECT_ROOT` como ponto de referência global para todos os caminhos do projeto.

#### 🧩 2. Validação da estrutura `utils/`  
- Verifica se existem a pasta `utils/` e o arquivo `__init__.py` (criado automaticamente, se ausente).  
- Garante que o pacote seja reconhecido pelo Python, permitindo importar `utils.utils_data` sem erros.

#### 🧱 3. Registro no `sys.path`  
- Adiciona o diretório raiz (`PROJECT_ROOT`) ao `sys.path`, habilitando o uso de módulos internos em qualquer ambiente.  
- Exibe o caminho detectado e o status do registro.

#### ♻️ 4. Importação e recarregamento das utilidades  
- Importa o módulo `utils.utils_data` e o recarrega via `importlib.reload`.  
- Essa abordagem assegura que o notebook use **sempre a versão mais recente** do arquivo durante o desenvolvimento iterativo.

#### 🧾 5. Inicialização do sistema de logs  
- Cria um log unificado em `reports/data_preparation.log`.  
- Cada etapa importante (carregamento, limpeza, exportação etc.) é registrada tanto no **console** quanto em **arquivo**, facilitando a rastreabilidade do processo.

#### ⚙️ 6. Carregamento das configurações globais  
- Lê os parâmetros de `config/defaults.json` e, se existir, aplica substituições de `config/local.json`.  
- Esses parâmetros controlam o comportamento de cada etapa (tratamento de nulos, tipagem, outliers, encoding etc.).

#### 📂 7. Resolução de diretórios e arquivos padrão  
- Garante a existência das pastas principais:  
  `data/raw`, `data/interim`, `data/processed`, `reports`, `artifacts`, `prints`, `dashboards`.  
- Define os arquivos de saída padrão (interim e processed), mantendo compatibilidade entre notebooks.

#### 🔄 8. Ambiente reprodutível e consistente  
- Define a semente aleatória (`RANDOM_SEED = 42`) para resultados reprodutíveis.  
- Ajusta as opções de exibição do pandas (`max_columns` e `display.width`) para uma visualização mais confortável.

---

### ✅ Resultado esperado

Ao final desta célula:

- O projeto é **reconhecido automaticamente**, independentemente do ambiente de execução.  
- O módulo `utils_data.py` é importado com todas as funções utilitárias disponíveis.  
- As pastas e arquivos padrão são criados e configurados.  
- O log central começa a registrar todas as ações executadas nas próximas etapas.  

---

> 💡 **Resumo:** Este design modular permite que o mesmo notebook seja executado em qualquer repositório que siga a estrutura do template, sem ajustes manuais de caminho ou imports.  


In [66]:
# -*- coding: utf-8 -*-
# Bootstrap do projeto — encontra a raiz, injeta no sys.path e garante utils/__init__.py

from IPython.display import display
from pathlib import Path
import sys
import logging
from datetime import datetime
import importlib
import pandas as pd
import random, numpy as np
import json

# 0) Raiz do projeto
PROJECT_ROOT = ud.ensure_project_root()

def _find_up(relative_path: str, start: Path | None = None) -> Path | None:
    start = start or Path.cwd()
    rel = Path(relative_path)
    for base in (start, *start.parents):
        cand = base / rel
        if cand.exists():
            return cand
    return None

# =========================================================
# 🎲 Reprodutibilidade (seed global)
# =========================================================
RANDOM_SEED = config.get("random_seed", 42)

import numpy as np, random, os
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)

# (opcional) se usar PyTorch/TF no futuro:
# import torch
# torch.manual_seed(RANDOM_SEED)
# torch.cuda.manual_seed_all(RANDOM_SEED)



# 1) Descobrir a raiz a partir do config/defaults.json
_cfg = _find_up("config/defaults.json")
if _cfg is None:
    raise FileNotFoundError("config/defaults.json não encontrado. Confirme a estrutura do projeto.")
PROJECT_ROOT = _cfg.parent.parent.resolve()
print(f"[INFO] PROJECT_ROOT: {PROJECT_ROOT}")

# 2) Garantir pasta utils/ e __init__.py
UTILS_DIR = PROJECT_ROOT / "utils"
INIT_FILE = UTILS_DIR / "__init__.py"
if not UTILS_DIR.exists():
    raise ModuleNotFoundError(f"Pasta não encontrada: {UTILS_DIR} (crie 'utils' na raiz).")
if not INIT_FILE.exists():
    INIT_FILE.write_text("", encoding="utf-8")
    print(f"[INFO] Criado: {INIT_FILE}")

# 3) Injetar a raiz no sys.path
root_str = str(PROJECT_ROOT)
if root_str not in sys.path:
    sys.path.insert(0, root_str)
print(f"[INFO] sys.path ok. utils: {UTILS_DIR}")

# 4) Importar utils.utils_data
import utils.utils_data as ud
importlib.reload(ud)  # garante versão mais recente ao iterar no notebook

# 5) Configurar logging base do notebook
LOG_FILE = (PROJECT_ROOT / "reports" / "data_preparation.log")
LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.StreamHandler(sys.stdout), logging.FileHandler(LOG_FILE, encoding="utf-8")],
    force=True,
)
logger = logging.getLogger(__name__)
logger.info("Bootstrap concluído.")

# 6) Carregar configurações
config = ud.load_config(PROJECT_ROOT / "config" / "defaults.json",
                        PROJECT_ROOT / "config" / "local.json")

paths = ud.resolve_n1_paths(PROJECT_ROOT)             

# 7) Seed e display (substitui ud.set_random_seed / ud.set_display)
random.seed(42)
np.random.seed(42)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 120)

logger.info("Config carregada e paths resolvidos.")


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 04:59:57,546 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] sys.path ok. utils: C:\Users\fabio\Projetos DEV\data projects\data-project-template\utils
2025-11-04 04:59:57,554 | INFO | Bootstrap concluído.
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 04:59:57,556 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 04:59:57,557 | INFO | Config carregada e paths resolvidos.


## 🔧 Configuração de Fontes

Nesta etapa são definidos os **arquivos de origem** que servirão de base para o projeto.  
Aqui você informa **quais datasets serão utilizados**, **em qual formato** estão (CSV ou Parquet) e, se houver mais de uma fonte, **como elas se relacionam**.

---

### 🗂️ 1. Exploração das fontes disponíveis

Antes de configurar o dicionário `SOURCES`, é possível listar os arquivos presentes no diretório `data/raw/`:

```python
from utils.utils_data import suggest_source_path, path_of

RAW_DIR = paths.raw_dir
suggest_source_path(RAW_DIR, pattern="*.csv")
```

Esse comando exibe uma tabela com os arquivos encontrados e gera uma **sugestão automática de caminho** para copiar e colar no `SOURCES`.  
Basta arrastar o arquivo para a pasta `data/raw/` e executar o bloco — o nome aparecerá pronto para uso.

---

### ⚙️ 2. Definição das fontes de dados (`SOURCES`)

- Cada entrada do dicionário `SOURCES` representa uma tabela nomeada.  
- A chave (ex.: `"main"`, `"dim_customers"`) identifica a tabela no projeto.  
- Cada item contém:
  - `path`: caminho do arquivo dentro de `data/raw/`.  
  - `format`: formato opcional (`"csv"` ou `"parquet"`). Se omitido, o formato é detectado automaticamente.  
  - `read_opts`: parâmetros de leitura personalizados (`encoding`, `sep`, `low_memory`, etc.).

Exemplo de estrutura:
```python
SOURCES = {
    "main": {
        "path": RAW_DIR / "dataset.csv",
        "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    },
    # Exemplo de segunda fonte em CSV:
    # "dim_customers": {
    #     "path": RAW_DIR / "exemplo.csv",
    #     "format": "csv"
    # }
    # Exemplo de fonte em Parquet:
    # "dim_customers": {
    #     "path": RAW_DIR / "customers.parquet",
    #     "format": "parquet"
    # }
}
```

---

### 🔗 3. Configuração opcional de junções (`MERGE_STEPS`)

O DataFrame principal é o definido em `MAIN_SOURCE` (geralmente `"main"`).  
Se existirem outras fontes, você pode configurá-las em `MERGE_STEPS` para realizar merges automáticos:

```python
MAIN_SOURCE = "main"
MERGE_STEPS = [
    # ("dim_customers", "left", "customer_id", "id"),
]
```

Cada item define:
- o nome da tabela secundária (`right_name`),
- o tipo de junção (`how`),
- e as chaves de correspondência (`left_on`, `right_on`).

---

### ✅ 4. Validação e feedback automático

Após definir as fontes, o bloco final realiza uma checagem simples:

- Confirma se o arquivo principal existe.  
- Detecta automaticamente o formato (`CSV` ou `Parquet`).  
- Exibe mensagens de status amigáveis:

```
✅ Fontes configuradas com sucesso.
→ Fonte principal: 'main'  | Arquivo: dataset.csv  | Formato detectado: CSV
→ Nenhum merge configurado (usando apenas a fonte principal).
```

---

> 💡 Resumo
>
>Essa abordagem torna a **configuração de fontes interativa, leve e flexível**:
>- Basta colocar novos arquivos em `data/raw/` e usar `suggest_source_path()` para gerar o caminho.  
>- Não há necessidade de alterar outros blocos do notebook — apenas o `SOURCES` e, se necessário, os merges.  
>- Ideal para projetos exploratórios e notebooks reutilizáveis em diferentes datasets.


In [67]:
import sys, importlib
sys.path.insert(0, r"C:\Users\fabio\Projetos DEV\data projects\data-project-template\utils")
import importlib, utils_data as ud
importlib.reload(ud)

RAW_DIR = paths.raw_dir
ud.suggest_source_path(RAW_DIR, pattern="*.csv")

[INFO] [suggest_source_path] 1 arquivo(s) encontrados; exibindo 1.
2025-11-04 04:59:59,213 | INFO | [suggest_source_path] 1 arquivo(s) encontrados; exibindo 1.


,path,name,size_bytes,modified
0,C:\Users\fabio\Projetos DEV\data projects\data...,dataset.csv,977501,2019-09-27T19:30:08


In [68]:
# Obs: só alterar este bloco quando precisar mudar os arquivos ou as chaves de junção.

SOURCES = {
    # nome_da_tabela: {path, (opcional) format: 'csv'|'parquet', (opcional) read_opts}
    "main": {
        "path": RAW_DIR / "dataset.csv",
        # "format": "csv",  # se omitir, detecta pelo sufixo
        "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    },
    # Exemplo de segunda fonte em CSV:
    # "dim_customers": {
    #   "path": RAW_DIR / "exemplo.csv",
    #   "format": "csv",  # se omitir, detecta pelo sufixo
    #   "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    # }
    
    # Exemplo de fonte em Parquet:
    # "dim_customers": {
    #     "path": RAW_DIR / "customers.parquet",
    #     "format": "parquet"
    # }
}

# Plano de merges (opcional):
# Cada item é um passo: (right_name, how, left_on, right_on)
# O DataFrame base será o SOURCES[MAIN_SOURCE]
MAIN_SOURCE = "main"
MERGE_STEPS = [
    # ("dim_customers", "left", "customer_id", "id"),
]

# Validação
main_path = SOURCES[MAIN_SOURCE]["path"]
main_format = SOURCES[MAIN_SOURCE].get("format") or main_path.suffix.lstrip(".").lower()
print(f"✅ Fontes configuradas com sucesso.")

print(
    f"→ Fonte principal: '{MAIN_SOURCE}'  "
    f"| Arquivo: {main_path.name}  "
    f"| Formato detectado: {main_format.upper()}"
)
if MERGE_STEPS:
    print(f"→ {len(MERGE_STEPS)} etapa(s) de merge configuradas.")
else:
    print("→ Nenhum merge configurado (usando apenas a fonte principal).")

✅ Fontes configuradas com sucesso.
→ Fonte principal: 'main'  | Arquivo: dataset.csv  | Formato detectado: CSV
→ Nenhum merge configurado (usando apenas a fonte principal).


## 📥 Ingestão & Visão Rápida

Nesta etapa, os **datasets definidos em `SOURCES`** são carregados, inspecionados e validados antes de iniciar o tratamento dos dados.  
O objetivo é garantir que as fontes estejam corretamente lidas e que o DataFrame principal (`df`) esteja pronto para seguir no pipeline.

---

### ⚙️ 1. Carregamento das fontes

Cada item de `SOURCES` é percorrido para:
- Ler o arquivo indicado (`path`) conforme o formato (`csv` ou `parquet`);  
- Aplicar as opções de leitura (`read_opts`) definidas no dicionário;  
- Armazenar o DataFrame em `tables[name]`, onde `name` é a chave de identificação da fonte.

A função utilizada é `ud.load_table_simple()`, que automaticamente:
- Detecta o formato pelo sufixo (se não informado);
- Aplica `read_opts` de forma segura;
- Exibe logs amigáveis durante o carregamento.

Após cada leitura, é exibido um **resumo rápido** da fonte carregada, com:
- Número de linhas e colunas;
- Tipos de dados;
- Uso de memória;
- E um relatório de valores nulos (`ud.missing_report()`).

---

### 🔗 2. Definição do DataFrame base e junções opcionais

O dataset principal é definido pela variável `MAIN_SOURCE`.  
Caso existam outras fontes configuradas em `MERGE_STEPS`, o notebook aplica as junções automaticamente via `ud.merge_chain()` — garantindo que cada passo do merge seja logado e validado.

Se nenhuma junção for configurada, o notebook utiliza apenas a fonte principal.

---

### 📊 3. Visão geral final do DataFrame

Após a etapa de ingestão e merges, o notebook gera uma visão consolidada do dataset final (`df`) que seguirá no pipeline.  
São exibidos:
- Estrutura geral (`shape`, colunas, tipos);
- Quantidade de valores nulos por coluna;
- E um log detalhado gravado em `reports/data_preparation.log`.

---

### 🧩 Exemplo de estrutura do processo

```python
tables = {}
for name, cfg in SOURCES.items():
    path = cfg["path"]
    fmt = cfg.get("format")
    read_opts = cfg.get("read_opts", {})
    df_src = ud.load_table_simple(path, fmt, **read_opts)
    tables[name] = df_src

    print(f"\n=== {name} ===")
    ov = ud.basic_overview(df_src)
    print(json.dumps(ov, indent=2, ensure_ascii=False))
    display(ud.missing_report(df_src).head(20))

# Base principal e merges
df = tables[MAIN_SOURCE]
if MERGE_STEPS:
    df = ud.merge_chain(df, tables, MERGE_STEPS)
else:
    print(f"[INFO] Usando df base: '{MAIN_SOURCE}' (sem merges).")

# Overview final
overview = ud.basic_overview(df)
logger.info(json.dumps(overview, indent=2, ensure_ascii=False))
display(ud.missing_report(df).head(20))
```

---

> 💡 **Resumo:**  
> - Cada fonte de dados é carregada automaticamente com base na configuração do dicionário `SOURCES`.  
> - A função `ud.load_table_simple()` lida tanto com arquivos CSV quanto Parquet e detecta o formato automaticamente.  
> - O dicionário `tables` mantém todas as fontes acessíveis pelo nome definido (ex.: `"main"`, `"dim_customers"`).  
> - A variável `df` representa o dataset principal que seguirá pelas próximas etapas do pipeline.  
> - Logs e relatórios são salvos para garantir transparência e rastreabilidade do processo.


In [69]:
import sys, importlib
sys.path.insert(0, r"C:\Users\fabio\Projetos DEV\data projects\data-project-template\utils")

import utils_data as ud
importlib.reload(ud)

print(ud.__file__)
print(ud.UTILS_DATA_VERSION)  # deve mostrar 1.1.1


# 1) Carregar todas as fontes
tables = {}
for name, cfg in SOURCES.items():
    path = cfg["path"]
    fmt = cfg.get("format")              # se None, detecta pelo sufixo
    read_opts = cfg.get("read_opts", {}) # ex.: sep/encoding/low_memory para CSV
    df_src = ud.load_table_simple(path, fmt, **read_opts)  # agora com prefixo ud.
    tables[name] = df_src

    # visão rápida por fonte
    print(f"\n=== {name} ===")
    ov = ud.basic_overview(df_src)
    print(json.dumps(ov, indent=2, ensure_ascii=False))
    display(ud.missing_report(df_src).head(20))

# 2) Definir df base e aplicar merges (se configurado)
if MAIN_SOURCE not in tables:
    raise KeyError(f"MAIN_SOURCE '{MAIN_SOURCE}' não encontrado. Fontes disponíveis: {list(tables.keys())}")

df = tables[MAIN_SOURCE]
if MERGE_STEPS:
    df = ud.merge_chain(df, tables, MERGE_STEPS)
    print("\n=== Visão geral (df merged) ===")
else:
    print(f"\n[INFO] Usando df base: '{MAIN_SOURCE}' (sem merges).")

# 3) Overview final do df que segue no pipeline
overview = ud.basic_overview(df)
logger.info(json.dumps(overview, indent=2, ensure_ascii=False))
print(json.dumps(overview, indent=2, ensure_ascii=False))
display(ud.missing_report(df).head(20))


C:\Users\fabio\Projetos DEV\data projects\data-project-template\utils\utils_data.py
1.2.2-merged

=== main ===
{
  "rows": 7043,
  "cols": 21,
  "dtypes": {
    "customerID": "object",
    "gender": "object",
    "SeniorCitizen": "int64",
    "Partner": "object",
    "Dependents": "object",
    "tenure": "int64",
    "PhoneService": "object",
    "MultipleLines": "object",
    "InternetService": "object",
    "OnlineSecurity": "object",
    "OnlineBackup": "object",
    "DeviceProtection": "object",
    "TechSupport": "object",
    "StreamingTV": "object",
    "StreamingMovies": "object",
    "Contract": "object",
    "PaperlessBilling": "object",
    "PaymentMethod": "object",
    "MonthlyCharges": "float64",
    "TotalCharges": "object",
    "Churn": "object"
  },
  "memory_mb": 6.821
}


,column,missing_count,missing_pct
0,customerID,0,0.0
1,DeviceProtection,0,0.0
2,TotalCharges,0,0.0
3,MonthlyCharges,0,0.0
4,PaymentMethod,0,0.0
5,PaperlessBilling,0,0.0
6,Contract,0,0.0
7,StreamingMovies,0,0.0
8,StreamingTV,0,0.0
9,TechSupport,0,0.0



[INFO] Usando df base: 'main' (sem merges).
2025-11-04 05:00:00,868 | INFO | {
  "rows": 7043,
  "cols": 21,
  "dtypes": {
    "customerID": "object",
    "gender": "object",
    "SeniorCitizen": "int64",
    "Partner": "object",
    "Dependents": "object",
    "tenure": "int64",
    "PhoneService": "object",
    "MultipleLines": "object",
    "InternetService": "object",
    "OnlineSecurity": "object",
    "OnlineBackup": "object",
    "DeviceProtection": "object",
    "TechSupport": "object",
    "StreamingTV": "object",
    "StreamingMovies": "object",
    "Contract": "object",
    "PaperlessBilling": "object",
    "PaymentMethod": "object",
    "MonthlyCharges": "float64",
    "TotalCharges": "object",
    "Churn": "object"
  },
  "memory_mb": 6.821
}
{
  "rows": 7043,
  "cols": 21,
  "dtypes": {
    "customerID": "object",
    "gender": "object",
    "SeniorCitizen": "int64",
    "Partner": "object",
    "Dependents": "object",
    "tenure": "int64",
    "PhoneService": "object",

,column,missing_count,missing_pct
0,customerID,0,0.0
1,DeviceProtection,0,0.0
2,TotalCharges,0,0.0
3,MonthlyCharges,0,0.0
4,PaymentMethod,0,0.0
5,PaperlessBilling,0,0.0
6,Contract,0,0.0
7,StreamingMovies,0,0.0
8,StreamingTV,0,0.0
9,TechSupport,0,0.0


## 🔖 Catálogo de DataFrames + df “ativo”

Esta etapa cria um **catálogo centralizado de DataFrames** para gerenciar facilmente todas as tabelas carregadas e derivadas ao longo do projeto.  
O objetivo é permitir que diferentes versões e transformações dos dados sejam armazenadas, nomeadas e acessadas de forma organizada e reprodutível.

---

### ⚙️ 1. Inicialização do catálogo (`TableStore`)

A classe `TableStore` (definida em `utils/utils_data.py`) atua como um **repositório de DataFrames nomeados**.  
Ela é inicializada com o dicionário `tables` (criado na etapa de *Ingestão & Visão Rápida*) e define a tabela principal (`MAIN_SOURCE`) como **tabela ativa**.

Durante a inicialização, o notebook exibe uma mensagem de sucesso semelhante a esta:

```
✅ Catálogo de DataFrames inicializado com sucesso.
→ Total de tabelas carregadas: 1
→ Tabela ativa: 'main'  | Shape: 7032 linhas × 21 colunas
```

E apresenta uma prévia do inventário atual:

| name | rows | cols | memory_mb |
|------|------|-------|-----------|
| main | 7032 | 21    | 1.2       |

---

### 📊 2. Definição do DataFrame ativo (`df`)

A variável `df` recebe a tabela atual através de `T.get()`.  
Esse `df` passa a representar o **DataFrame padrão** que seguirá nas próximas etapas do pipeline (limpeza, tipagem e transformação).

---

### 🧩 3. Gerenciamento de múltiplas tabelas

Novos DataFrames podem ser adicionados ao catálogo a qualquer momento, com nomes descritivos e controle de versão.  
O catálogo também permite alternar entre tabelas e listar todas as disponíveis.

**Principais comandos:**

```python
T.add("churn_raw", df, set_current=True)           # adiciona e define como atual
df = T.use("churn_raw")                            # alterna o df ativo
T.add("features_v1", engenharia_de_atributos(df))  # armazena uma nova derivação
df_features = T["features_v1"]                     # acesso direto por nome
display(T.list())                                  # exibe o inventário completo
```

---

### 🧠 4. Benefício prático

- Mantém o notebook limpo e evita sobrescrever DataFrames importantes.  
- Facilita o reuso e o rastreamento das versões intermediárias dos dados.  
- Ideal para projetos com múltiplas fontes, transformações paralelas ou comparações entre conjuntos tratados.  
- Fornece feedback visual imediato sobre o estado do pipeline, tornando o fluxo mais transparente e interativo.

---

> 💡 **Resumo:**  
> O `TableStore` funciona como uma **memória estruturada de DataFrames**, permitindo adicionar, alternar, versionar e consultar tabelas de forma controlada.  
> A variável `df` representa sempre a **tabela ativa** atual, garantindo consistência nas etapas seguintes do pipeline e visibilidade sobre o progresso do processamento.


In [70]:
from utils.utils_data import TableStore

# 1- Inicializa catálogo com as fontes lidas ('tables') e define a base como atual
T = TableStore(initial=tables, current=MAIN_SOURCE)

# 2 - Conveniência: df = tabela atual (boa prática para seguir no pipeline)
df = T.get()

# 3 - Feedback visual de sucesso
print(f"✅ Catálogo de DataFrames inicializado com sucesso.")
print(f"→ Total de tabelas carregadas: {len(tables)}")
print(f"→ Tabela ativa: '{T.current}'  | Shape: {df.shape[0]} linhas × {df.shape[1]} colunas")

# 4️4 - Exibe uma prévia do catálogo
display(T.list().head())

# --- Guia rápido  ---
# T.add("churn_raw", df, set_current=True)          # cadastra e ativa
# df = T.use("churn_raw")                           # muda o atual e retorna df
# T.add("features_v1", engenharia_de_atributos(df)) # salva uma derivação
# df_features = T["features_v1"]                    # acesso direto por nome
# display(T.list())                                 # inventário das tabelas

✅ Catálogo de DataFrames inicializado com sucesso.
→ Total de tabelas carregadas: 1
→ Tabela ativa: 'main'  | Shape: 7043 linhas × 21 colunas


,name,rows,cols,memory_mb,current
0,main,7043,21,6.821,True


In [71]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 🧪 Qualidade & Tipagem

Nesta etapa o notebook realiza a **padronização estrutural e tipagem automática dos dados**, garantindo que o DataFrame principal (`df`) siga para as próximas fases **coerente, limpo e otimizado em memória**.  
É uma das fases mais críticas do pipeline, pois evita que inconsistências de tipo e formato prejudiquem as análises posteriores ou a modelagem.

### 📋 O que é feito aqui

1. **Remoção de espaços em branco (`strip_whitespace`)**
   - Elimina espaços extras no início e no fim de valores textuais.  
   - Evita divergências em comparações e agrupamentos (ex.: `"Yes "` ≠ `"Yes"`).  
   - Essa operação é **idempotente**: se os dados já estiverem limpos, não altera nada.

2. **Conversão inteligente de valores numéricos (`infer_numeric_like`)**
   - Detecta colunas com valores *aparentemente numéricos*, mas armazenados como texto (ex.: `"R$ 120,00"` ou `"1.234,56"`).  
   - Aplica heurísticas para remover símbolos, normalizar separadores e converter apenas quando a proporção de conversão for suficiente (`min_ratio = 0.9`).  
   - Evita conversões indevidas (ex.: IDs) por meio de uma lista de exclusão (`blacklist`).  
   - Gera o relatório **`cast_report`**, registrando:
     - Coluna analisada  
     - Ação tomada (convertida, ignorada ou parcial)  
     - Taxa de sucesso e total de valores convertidos.

3. **Otimização de tipos numéricos (`reduce_memory_usage`)**
   - Converte automaticamente `int64` → `int32` e `float64` → `float32`, reduzindo o consumo de memória sem alterar os valores.  
   - O log mostra a diferença antes e depois (ex.: `Memory reduced: 7.76MB → 6.21MB`).

4. **Remoção de duplicatas (`deduplicate_rows`)**
   - Verifica e elimina registros repetidos conforme a configuração (`subset`, `keep`).  
   - Caso encontre duplicatas, gera um relatório detalhado e salva o log em `reports/duplicates.csv`.  
   - Exibe uma amostra das duplicatas detectadas, se houver.

5. **Relatórios e feedback automático**
   - Ao final da execução, exibe um resumo completo:  
     - Linhas e colunas antes/depois  
     - Variação de memória (`Δ MB`)  
     - Relatórios gerados (`cast_report`, `duplicates_report`, etc.)  
   - Todos os logs são gravados em `reports/data_preparation.log`.

---

> 💡 **Resumo:**  
> Esta célula garante que o dataset esteja **consistente, tipado e otimizado**, pronto para análises e modelagem.  
> Corrige formatações, converte números armazenados como texto, remove duplicatas e reduz o uso de memória — tudo com registros automáticos e relatórios salvos para auditoria.


In [72]:
# Qualidade & Tipagem (compatível com ambas as APIs)

df_before_shape = df.shape
mem_before = float(df.memory_usage(deep=True).sum() / (1024**2))

# ⚠️ Importante: NÃO passe paths.reports_dir como 'root', pois isso duplicará 'reports/reports'.
# Deixe o utils resolver a raiz do projeto internamente.
if hasattr(ud, "n1_quality_typing_dict"):
    out = ud.n1_quality_typing_dict(df, config)   # sempre retorna dict
    df  = out["df"]
    rep = out
else:
    df, rep = ud.n1_quality_typing(df, config)    # retorna (df, meta_dict)

mem_after  = float(df.memory_usage(deep=True).sum() / (1024**2))
delta_rows = df.shape[0] - df_before_shape[0]
delta_cols = df.shape[1] - df_before_shape[1]
delta_mem  = mem_after - mem_before

# Exibe relatórios úteis (se existirem)
cast_report = rep.get("cast_report") if isinstance(rep, dict) else None
if isinstance(cast_report, pd.DataFrame) and not cast_report.empty:
    print("📄 Conversões numéricas (amostra):")
    display(cast_report.head(20))

dups = rep.get("duplicates") if isinstance(rep, dict) else None
if isinstance(dups, pd.DataFrame) and not dups.empty:
    print("\n🔁 Duplicatas detectadas (amostra):")
    display(dups.head(10))
    dsum = rep.get("duplicates_summary") if isinstance(rep, dict) else None
    if isinstance(dsum, pd.DataFrame) and not dsum.empty:
        print("📊 Resumo por chave:")
        display(dsum.head(20))
else:
    print("\n✅ Nenhuma duplicidade encontrada segundo os critérios definidos.")

# Mensagem de sucesso consolidada
reported_keys = []
if isinstance(rep, dict):
    for k, v in rep.items():
        if isinstance(v, pd.DataFrame) and not v.empty:
            reported_keys.append(k)

print(
    f"\n✅ Qualidade & Tipagem concluído com sucesso!\n"
    f"→ Shape: {df_before_shape} → {df.shape} (Δlinhas={delta_rows:+}, Δcolunas={delta_cols:+})\n"
    f"→ Memória: {mem_before:.2f} MB → {mem_after:.2f} MB (Δ={delta_mem:+.2f} MB)\n"
    f"→ Relatórios gerados: {', '.join(reported_keys) if reported_keys else '—'}"
)


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:01,947 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:01,962 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [manifest] step='n1_quality_typing:start' registrado.
2025-11-04 05:00:01,967 | INFO | [manifest] step='n1_quality_typing:start' registrado.
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,145 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,149 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-

,column,converted_non_null,introduced_nans,dtype_after
16,TotalCharges,7032,11,float64
0,customerID,0,7043,object
1,gender,0,7043,object
15,PaymentMethod,0,7043,object
14,PaperlessBilling,0,7043,object
13,Contract,0,7043,object
12,StreamingMovies,0,7043,object
11,StreamingTV,0,7043,object
10,TechSupport,0,7043,object
9,DeviceProtection,0,7043,object



✅ Nenhuma duplicidade encontrada segundo os critérios definidos.

✅ Qualidade & Tipagem concluído com sucesso!
→ Shape: (7043, 21) → (7043, 21) (Δlinhas=+0, Δcolunas=+0)
→ Memória: 6.82 MB → 6.51 MB (Δ=-0.31 MB)
→ Relatórios gerados: cast_report, df


## 🧼 Padronização Categórica

Nesta etapa ocorre a **padronização global de rótulos textuais** em colunas categóricas, corrigindo inconsistências, diferenças de capitalização e valores redundantes (como `"No internet service"` → `"No"`).  
O objetivo é **garantir uniformidade sem alterar o significado dos dados**, preparando o dataset para codificação e modelagem.

---

### 📋 O que é feito aqui

1. **Seleção automática das colunas categóricas**  
   - São processadas todas as colunas com tipo `object`, `string` ou `category`, exceto aquelas listadas em `exclude` (ex.: `"customerID"`).

2. **Limpeza e normalização de forma**  
   - Remove espaços extras (`trim`) e consolida espaços duplos (`collapse_ws`).  
   - Corrige capitalização conforme o parâmetro `case` (`"title"`, `"lower"`, `"upper"`, `"none"`).  
   - Opcionalmente remove acentos (`strip_accents`).

3. **Substituições e valores nulos**  
   - Aplica um mapa global de substituições (`global_map`) para corrigir padrões conhecidos (ex.: `"No internet service"` → `"No"`).  
   - Permite também um mapa específico por coluna (`per_column_map`), quando necessário.  
   - Converte valores definidos em `null_values` (ex.: `"n/a"`, `"na"`, `"-"`) em `NaN`.

4. **Conversão opcional para categoria**  
   - Se `cast_to_category=True`, converte colunas textuais em `category`, otimizando uso de memória.

5. **Geração de relatório e registro automático**  
   - Cria o arquivo `cat_normalization.csv` dentro da pasta `reports/`, com:  
     - Nome da coluna  
     - Amostras antes/depois  
     - Quantidade de alterações (`changes`)  
     - Número de categorias únicas antes/depois  
   - As informações também são registradas no log e adicionadas ao `manifest.json`.

---

### ⚙️ Configuração (`CAT_NORM_CFG`)

A padronização é controlada por um dicionário de configuração completo:

```python
CAT_NORM_CFG = {
    "enabled": True,
    "exclude": ["customerID"],
    "case": "title",
    "strip_accents": True,
    "collapse_ws": True,
    "trim": True,
    "global_map": {
        "No internet service": "No",
        "No phone service": "No",
        "n/a": "No",
        "none": "No"
    },
    "null_values": ["", "na", "n/a", "-"],
    "cast_to_category": False
}
```

Exemplo de mapa específico por coluna:
```python
"per_column_map": {
    "InternetService": {"Dsl": "DSL", "Fiber Optic": "Fiber Optic"}
}
```

---

> 💡 **Resumo:**  
> Esta célula realiza uma **padronização inteligente e auditável** das colunas categóricas — aplicando limpeza, substituições e normalização de forma sistemática.  
> O processo é controlado via configuração (`CAT_NORM_CFG`), gera um relatório automático e mantém registro completo no log e no `manifest.json`, garantindo **clareza, rastreabilidade e consistência** no tratamento dos dados.


In [73]:
# Config genérica de normalização (ajuste à vontade)
CAT_NORM_CFG = {
    "enabled": True,
    "exclude": ["customerID"],          # nunca normalizar IDs
    "case": "title",                    # "lower" | "upper" | "title"
    "strip_accents": True,
    "collapse_ws": True,
    "trim": True,
    "global_map": {
        "No internet service": "No",
        "No phone service": "No",
        "n/a": "No",
        "none": "No"
    },
    "null_values": ["", "na", "n/a", "-"],
    "cast_to_category": False,
    # "per_column_map": {"PaperlessBilling": {"sim": "Yes", "nao": "No"}}
}

# Guarda métricas antes/depois
_before_shape = df.shape
_before_mem   = float(df.memory_usage(deep=True).sum() / (1024**2))

# Caminho de relatório
cat_report_path = paths.reports_dir / "cat_normalization.csv"

# Tentativa 1: usar a função avançada (com cfg e relatório)
_used_fallback = False
try:
    df_norm, cat_norm_report = ud.normalize_categories(
        df,
        cfg=CAT_NORM_CFG,
        report_path=cat_report_path  # atenção: é um caminho completo; essa função já lida com Path
    )
except TypeError:
    # Tentativa 2: fallback para versão simples do utils (sem cfg, sem report interno)
    _used_fallback = True

    # Limita às colunas de texto e aplica "exclude" da config
    text_cols = [c for c in df.columns if df[c].dtype == "object" or pd.api.types.is_string_dtype(df[c])]
    target_cols = [c for c in text_cols if c not in set(CAT_NORM_CFG.get("exclude", []))]

    df_before = df.copy()

    # Chamada da versão simples
    df_norm = ud.normalize_categories(
        df,
        cols=target_cols,
        case=CAT_NORM_CFG.get("case", "lower"),
        trim=CAT_NORM_CFG.get("trim", True),
        strip_accents=CAT_NORM_CFG.get("strip_accents", True),
    )

    # Se a função simples retornar só o df, mantenha:
    if isinstance(df_norm, tuple):
        # alguma variante pode retornar (df, report); normaliza para df apenas
        df_norm = df_norm[0]

    # Constrói um relatório equivalente (diferenças por coluna)
    changes = []
    for c in target_cols:
        changed = (df_before[c].astype(str) != df_norm[c].astype(str)).sum()
        if changed > 0:
            changes.append({"column": c, "changed": int(changed)})

    cat_norm_report = pd.DataFrame(changes).sort_values("changed", ascending=False).reset_index(drop=True)
    cat_report_path.parent.mkdir(parents=True, exist_ok=True)
    cat_norm_report.to_csv(cat_report_path, index=False, encoding="utf-8")

# Atualiza o df do pipeline
df = df_norm

# Métricas pós
_after_mem   = float(df.memory_usage(deep=True).sum() / (1024**2))
_delta_rows  = df.shape[0] - _before_shape[0]
_delta_cols  = df.shape[1] - _before_shape[1]
_delta_mem   = _after_mem - _before_mem

# Exibição do relatório
if isinstance(cat_norm_report, pd.DataFrame) and not cat_norm_report.empty:
    print("📑 Relatório de padronização (top 20 por mudanças):")
    display(cat_norm_report.head(20))
else:
    print("✅ Nenhuma mudança significativa detectada nas colunas categóricas.")

# Mensagem consolidada
print(
    f"\n✅ Padronização Categórica concluída "
    f"{'(fallback simples)' if _used_fallback else '(função avançada)'}\n"
    f"→ Shape: {_before_shape} → {df.shape} (Δlinhas={_delta_rows:+}, Δcolunas={_delta_cols:+})\n"
    f"→ Memória: {_before_mem:.2f} MB → {_after_mem:.2f} MB (Δ={_delta_mem:+.2f} MB)\n"
    f"→ Relatório salvo em: {cat_report_path}"
)


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,583 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,595 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [manifest] step='save_report_df' registrado.
2025-11-04 05:00:02,599 | INFO | [manifest] step='save_report_df' registrado.
📑 Relatório de padronização (top 20 por mudanças):


,column,changed
0,Contract,7043
1,PaymentMethod,7043
2,InternetService,5517
3,OnlineSecurity,1526
4,OnlineBackup,1526
5,DeviceProtection,1526
6,TechSupport,1526
7,StreamingTV,1526
8,StreamingMovies,1526
9,MultipleLines,682



✅ Padronização Categórica concluída (função avançada)
→ Shape: (7043, 21) → (7043, 21) (Δlinhas=+0, Δcolunas=+0)
→ Memória: 6.51 MB → 6.51 MB (Δ=+0.00 MB)
→ Relatório salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\cat_normalization.csv


## 🩹 Tratamento de Valores Faltantes

Nesta etapa são tratadas as **ausências de dados** (valores nulos ou `NaN`) para garantir que o DataFrame siga consistente para as próximas fases do pipeline.  
O objetivo é **preencher ou sinalizar valores faltantes** de forma controlada, preservando a integridade estatística das colunas e registrando quais linhas foram modificadas.

### 📋 O que é feito aqui

1. **Geração de relatório inicial**
   - Exibe um diagnóstico das colunas com valores ausentes, mostrando a proporção (`missing_rate`) e o total (`missing_count`) de registros nulos.
   - Essa visualização ajuda a decidir se o tratamento será simples ou se exigirá técnicas específicas.

2. **Aplicação da estratégia de imputação**
   - O comportamento é controlado pela chave `missing_strategy` do arquivo de configuração:
     - `"simple"` → aplica a função `simple_impute_with_flags()`
     - `"advanced"` → reservado para métodos personalizados (ex.: KNN, regressão, interpolação)
   - A estratégia **simples** executa:
     - Substituição de valores nulos **numéricos** pela **mediana** da coluna.  
     - Substituição de valores nulos **categóricos** pela **moda** (valor mais frequente).  
   - Durante a imputação, são criadas **colunas de flag** no formato `was_imputed_<coluna>`, indicando quais linhas foram alteradas.

3. **Relatório final de verificação**
   - Após o preenchimento, é exibido novamente o relatório de faltantes para confirmar se todas as lacunas foram resolvidas.
   - Caso ainda existam colunas críticas, o notebook pode ser ajustado para aplicar métodos mais avançados.

### Exemplo de flag gerada:
| customerID | TotalCharges | was_imputed_TotalCharges |
|-------------|--------------|--------------------------|
| 7590-VHVEG  | 29.85        | False                   |
| 9237-HQITU  | 1840.75      | True                    |

Linhas com `True` indicam que o valor original estava ausente e foi imputado automaticamente.

> 💡 **Resumo:**  
> Esta célula identifica e corrige valores faltantes, aplicando regras de imputação simples e registrando onde cada substituição ocorreu.  
> Isso garante **transparência, rastreabilidade e controle de qualidade**, permitindo filtrar posteriormente apenas os dados considerados consistentes.


In [74]:
# === N1 · Tratamento de Valores Faltantes (compacto) =========================
import importlib, utils.utils_data as ud
importlib.reload(ud)

# Garante df como DataFrame caso alguma etapa tenha retornado tupla
df = ud.coerce_df(df)

res = ud.handle_missing_step(
    df,
    config=config,                     # usa config["missing"] ou chaves antigas
    save_reports=True,                 # salva before/after em reports/missing/
    prefer="auto"                      # "auto" tenta strategy do config e faz fallback p/ simple
)

df = res["df"]

print("\n✅ Tratamento de faltantes concluído!")
print(f"→ Estratégia final: {res['strategy']}")
print(f"→ Colunas imputadas (amostra): {res.get('imputed_cols', [])[:10] or '—'}")

from IPython.display import display
print("\nRelatório de faltantes (antes):")
display(res["before"].head(20))

print("\nRelatório de faltantes (depois):")
display(res["after"].head(20))


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,650 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,653 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,665 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [manifest] step='save_report_df' registrado.
2025-11-04 05:00:02,669 | INFO | [manifest] step='save_report_df' registrado.
[INFO] [report] salvo: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\missing\before.csv (21 linhas)
2025-11-04 05:00:02,671 | INFO | [report] salvo: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\missing\before.csv (21 linhas)
[INF

,column,missing_count,missing_pct
0,TotalCharges,11,0.16
1,customerID,0,0.00
2,DeviceProtection,0,0.00
3,MonthlyCharges,0,0.00
4,PaymentMethod,0,0.00
5,PaperlessBilling,0,0.00
6,Contract,0,0.00
7,StreamingMovies,0,0.00
8,StreamingTV,0,0.00
9,TechSupport,0,0.00



Relatório de faltantes (depois):


,column,missing_count,missing_pct
0,customerID,0,0.0
1,InternetService_was_missing,0,0.0
2,MonthlyCharges_was_missing,0,0.0
3,TotalCharges_was_missing,0,0.0
4,customerID_was_missing,0,0.0
5,gender_was_missing,0,0.0
6,Partner_was_missing,0,0.0
7,Dependents_was_missing,0,0.0
8,PhoneService_was_missing,0,0.0
9,MultipleLines_was_missing,0,0.0


## 🚩 Detecção de Outliers

Esta etapa identifica **valores atípicos** nas colunas numéricas, adicionando colunas de flag (`*_is_outlier`) que indicam quais registros se desviam do padrão estatístico esperado.  
Essas colunas servem para **inspeção e auditoria**, sem alterar ou remover dados — a decisão de tratamento posterior (ajuste, exclusão ou manutenção) é **de negócio**.

### ⚙️ Como funciona

1. **Verifica a configuração**
   - A execução depende da flag `detect_outliers` no arquivo `config/defaults.json`.  
     Se estiver definida como `true`, a detecção é realizada.

2. **Seleciona o método estatístico**
   - `"iqr"` → usa o **Intervalo Interquartil (Interquartile Range)**:
     - Calcula Q1 (25º percentil) e Q3 (75º percentil)
     - Define limites:  
       Inferior = Q1 − 1.5 × IQR  
       Superior = Q3 + 1.5 × IQR
     - Valores fora desse intervalo são marcados como outliers.
   - `"zscore"` → usa o **desvio-padrão (Z-Score)**:
     - Calcula a média (μ) e o desvio-padrão (σ)
     - Para cada valor, obtém `z = (x − μ) / σ`
     - Valores com |z| > 3 são considerados outliers.

3. **Criação de colunas de flag**
   - Para cada variável numérica analisada, é criada uma nova coluna:
     ```
     <coluna>_is_outlier
     ```
   - O valor será:
     - `True` → registro identificado como outlier  
     - `False` → registro dentro do intervalo esperado

4. **Registro no log**
   - Ao final, o sistema registra no log quantas colunas de flag foram criadas:
     ```
     Outlier flags created: 4
     ```
     Significa que 4 colunas numéricas foram analisadas e receberam suas respectivas flags.

### Exemplo de resultado

| tenure | MonthlyCharges | TotalCharges | tenure_is_outlier | MonthlyCharges_is_outlier | TotalCharges_is_outlier |
|--------|----------------|--------------|-------------------|----------------------------|--------------------------|
| 5      | 35.50          | 190.00       | False             | False                      | False                    |
| 72     | 120.00         | 9999.99      | False             | **True**                   | **True**                 |

> 💡 **Resumo:**  
> Esta célula adiciona colunas de marcação para detectar valores atípicos de acordo com o método configurado.  
> Os dados originais são preservados, permitindo que a análise posterior defina se esses registros devem ser **mantidos, ajustados ou removidos**.


In [75]:
importlib.reload(ud)

cfg = ud.load_config()  # ou um dict manual

df2, out_info = ud.apply_outlier_flags(df, config=cfg)

print(out_info["created_flags"], "flags criadas")
display(pd.Series(out_info["counts"]).sort_values(ascending=False).head(20))

[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,909 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,925 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,928 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:02,940 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [manifest] step='save_report_df' registrado.
2025-11-04 05:00:02,946 | INFO | [manifest] step='save_report_df' registrado.
[INFO] [report] salvo: C:\Users\fabio\Projetos DEV\data projects\data-project-template

tenure_is_outlier            0
MonthlyCharges_is_outlier    0
TotalCharges_is_outlier      0
dtype: int64

In [76]:
display(df2.filter(like="_is_outlier").sum())

tenure_is_outlier            0
MonthlyCharges_is_outlier    0
TotalCharges_is_outlier      0
dtype: int64

## 🧬 Duplicidades

Remove **linhas duplicadas** do DataFrame para evitar contagens infladas, vieses e ruído em métricas.

### 📋 O que acontece aqui
- A duplicidade é verificada **linha a linha**, considerando todas as colunas por padrão.  
- A primeira ocorrência é mantida e as demais são removidas.

### 🔧 Opções (definidas no `config`)
- `deduplicate.subset`: lista de colunas que definem a chave de deduplicação.  
  *Ex.:* `["customerID"]` mantém apenas um registro por cliente.  
- `deduplicate.keep`: política de retenção — `"first"`, `"last"` ou `false` (remove todas as repetições).

### 🧠 Boas práticas
- Utilize `subset` quando a duplicidade for **conceitual** (ex.: mesma pessoa/pedido), não necessariamente toda a linha idêntica.
- Em datasets extensos, a deduplicação deve ser feita **após** o tratamento de nulos e padronização de texto, para evitar falsos positivos.

> 💡 **Resumo:** esta etapa garante um dataset **sem registros repetidos** segundo o critério configurado, mantendo a consistência das análises.


In [77]:
if cfg.get("deduplicate"):
    df = ud.deduplicate_rows(df, config=cfg)


[INFO] [deduplicate_rows] Removidas 0 duplicadas (7043 → 7043) | subset=ALL | keep=first
2025-11-04 05:00:03,361 | INFO | [deduplicate_rows] Removidas 0 duplicadas (7043 → 7043) | subset=ALL | keep=first


## 🛠️ Engenharia de Atributos

Criação de novas colunas que capturem **relações, proporções, categorias ou padrões** relevantes ao negócio.  
Essa etapa é inteiramente manual e depende do contexto do dataset.

### 💡 Exemplos:
- **Razões e proporções:** `TotalCharges / tenure` → gasto médio mensal.  
- **Flags categóricas:** `Contract == 'Month-to-month'` → 1 se contrato mensal.  
- **Contagens de serviços:** soma de colunas binárias (Yes/No).  

Essas novas features tornam as análises mais ricas e **melhoram a performance de modelos preditivos**.

---

### ⚙️ O que o bloco de código faz

- Verifica se `feature_engineering` está **habilitado no config**.  
- Aplica regras **automáticas seguras** (ex.: log1p em colunas com alta assimetria, razão de `TotalCharges / tenure`, flag `is_long_tenure_24m`, etc.).  
- Permite **regras personalizadas** via `config`, como:
  - `ratios`: lista de pares `[numerador, denominador, nome_da_nova_coluna]`
  - `log1p_cols`: colunas nas quais aplicar transformação logarítmica
  - `binaries`: limiares para gerar flags booleanas
  - `date_parts`: colunas de data das quais extrair partes (ano, mês, semana...)

Quando nenhuma regra é aplicável, o notebook retorna:

```
ℹ️ Nenhuma engenharia de atributos aplicada.
   - Verifique config['feature_engineering'].
   - Exemplos de regras: log1p_cols, ratios, binaries, date_parts.
   - Habilite enable_default_rules para heurísticas automáticas.
```

### 🧠 Boas práticas
- Use nomes **claros e descritivos** para novas features (`avg_charge_per_month`, `is_tenure_ge_12m`, etc.).  
- Evite criar colunas redundantes ou altamente correlacionadas.  
- Sempre documente cada feature gerada — especialmente se for usada em modelagem posterior.

> 💡 **Resumo:** esta etapa adiciona inteligência ao dataset, criando variáveis derivadas que capturam padrões implícitos e aumentam o poder explicativo dos modelos.


In [78]:
# --- Engenharia de Atributos -------------------------------------------------
# Regras opcionais no config:
# config.get("feature_engineering") pode ser:
#   - True/False
#   - dict com opções:
#       {
#         "enable_default_rules": True,
#         "log1p_cols": ["MonthlyCharges"],
#         "ratios": [["TotalCharges","tenure","avg_charge_per_month"]],  # num/den->nome
#         "binaries": [["tenure", 24, "is_long_tenure"]],                # col, limiar, nome
#         "date_parts": ["order_date"]                                   # extrai ano/mes/dia/semana
#       }


fe_cfg = config.get("feature_engineering", False)
if not fe_cfg:
    print("⚙️ FE desabilitada em config['feature_engineering'].")
else:
    created = []

    # helper p/ log
    def _log(msg):
        try:
            logger.info(msg)  # se você tiver logger configurado
        except:
            print(msg)

    # Normaliza config em dict
    if isinstance(fe_cfg, bool):
        fe_cfg = {"enable_default_rules": True}

    # -------------------- 1) Regras explícitas do config --------------------
    # (a) Razões numéricas definidas pelo usuário: [[num, den, nome], ...]
    for rule in fe_cfg.get("ratios", []) or []:
        if len(rule) == 3:
            num, den, out = rule
            if {num, den}.issubset(df.columns):
                denom = df[den].replace(0, np.nan)
                df[out] = df[num] / denom
                created.append(out)

    # (b) Log1p em colunas positivas
    for col in fe_cfg.get("log1p_cols", []) or []:
        if col in df.columns:
            pos = df[col] > 0
            if pos.any():
                out = f"{col}_log1p"
                df[out] = 0.0
                df.loc[pos, out] = np.log1p(df.loc[pos, col].astype(float))
                created.append(out)

    # (c) Binários por limiar: [[col, threshold, out_name], ...]
    for rule in fe_cfg.get("binaries", []) or []:
        if len(rule) == 3:
            col, thr, out = rule
            if col in df.columns:
                try:
                    df[out] = (pd.to_numeric(df[col], errors="coerce") >= float(thr))
                    created.append(out)
                except Exception:
                    pass

    # (d) Partes de data (ano, mês, dia, semana ISO)
    for col in fe_cfg.get("date_parts", []) or []:
        if col in df.columns and pd.api.types.is_datetime64_any_dtype(df[col]):
            for part, accessor in {
                "year": "year", "month": "month", "day": "day", "week": "isocalendar().week"
            }.items():
                out = f"{col}_{part}"
                try:
                    if part == "week":
                        df[out] = getattr(df[col].dt.isocalendar(), "week")
                    else:
                        df[out] = getattr(df[col].dt, part)
                    created.append(out)
                except Exception:
                    pass

    # -------------------- 2) Regras padrão (heurísticas seguras) ------------
    if fe_cfg.get("enable_default_rules", True):
        # (a) Telco clássico: cobrança média mensal estimada
        if {"TotalCharges", "tenure"}.issubset(df.columns):
            denom = pd.to_numeric(df["tenure"], errors="coerce").replace(0, np.nan)
            out = "avg_charge_per_month"
            df[out] = pd.to_numeric(df["TotalCharges"], errors="coerce") / denom
            created.append(out)

        # (b) Diferença entre o esperado (tenure*Monthly) e TotalCharges
        if {"MonthlyCharges", "tenure", "TotalCharges"}.issubset(df.columns):
            exp_total = (
                pd.to_numeric(df["MonthlyCharges"], errors="coerce") *
                pd.to_numeric(df["tenure"], errors="coerce")
            )
            out = "charge_gap"
            df[out] = pd.to_numeric(df["TotalCharges"], errors="coerce") - exp_total
            created.append(out)

        # (c) Limiar simples de fidelidade
        if "tenure" in df.columns:
            out = "is_long_tenure_24m"
            df[out] = (pd.to_numeric(df["tenure"], errors="coerce") >= 24)
            created.append(out)

        # (d) log1p automático em numéricas com valores > 0 e forte assimetria
        num_cols = df.select_dtypes(include=[np.number]).columns
        for col in num_cols:
            s = pd.to_numeric(df[col], errors="coerce")
            if (s > 0).sum() > 0:
                skew = s.skew(skipna=True)
                if pd.notna(skew) and abs(skew) > 1.0:
                    out = f"{col}_log1p"
                    # evita recriar se já existe por config
                    if out not in df.columns:
                        df[out] = np.log1p(s.clip(lower=0))
                        created.append(out)

    # -------------------- 3) Relato do que foi feito -------------------------
    if created:
        created = sorted(set(created))
        _log(f"[feature_engineering] Features criadas: {len(created)} -> {created[:8]}{'...' if len(created)>8 else ''}")
        # Visual rápido (limita para não poluir)
        display(df[created].head(10))
    else:
        _log("[feature_engineering] Nenhuma feature criada (sem regras aplicáveis para as colunas atuais).")
        print(
            "ℹ️ Nenhuma engenharia de atributos aplicada.\n"
            "   - Verifique `config['feature_engineering']`.\n"
            "   - Exemplos de regras: "
            "`log1p_cols`, `ratios`, `binaries`, `date_parts`.\n"
            "   - Habilite `enable_default_rules` para heurísticas automáticas."
        )


2025-11-04 05:00:07,374 | INFO | [feature_engineering] Features criadas: 6 -> ['SeniorCitizen_log1p', 'TotalCharges_was_missing_log1p', 'avg_charge_per_month', 'charge_gap', 'charge_gap_log1p', 'is_long_tenure_24m']


,SeniorCitizen_log1p,TotalCharges_was_missing_log1p,avg_charge_per_month,charge_gap,charge_gap_log1p,is_long_tenure_24m
0,0.0,0.0,29.850000,0.00,0.000000,False
1,0.0,0.0,55.573529,-46.80,0.000000,True
2,0.0,0.0,54.075000,0.45,0.371564,False
3,0.0,0.0,40.905556,-62.75,0.000000,True
4,0.0,0.0,75.825000,10.25,2.420368,False
5,0.0,0.0,102.562500,23.30,3.190476,False
6,0.0,0.0,88.609091,-10.80,0.000000,False
7,0.0,0.0,30.190000,4.40,1.686399,False
8,0.0,0.0,108.787500,111.65,4.724286,True
9,0.0,0.0,56.257258,6.65,2.034706,True


# 📅 Tratamento de Datas

Esta etapa detecta e converte colunas de datas de forma **automática e controlada**, criando **features temporais** úteis para análises e modelos preditivos.

---

## 📋 O que acontece aqui

### 🔹 Identificação de colunas de data

O sistema procura automaticamente colunas com nomes que contenham termos como  
`date`, `data`, `dt_`, `_dt`, ou `_date`.

Também é possível **forçar colunas específicas** definindo manualmente em:

```python
date_cfg["explicit_cols"] = ["StartDate", "EndDate"]
```

### 🔹 Conversão para datetime com auditoria

Cada coluna candidata é testada com diferentes formatos e tentativas de *parsing*.  
O relatório **`parse_report`** mostra:

| column      | parsed_ratio | converted |
|--------------|--------------|------------|
| order_date   | 1.00         | True       |
| start_date   | 0.35         | False      |

- **column:** nome da coluna testada  
- **parsed_ratio:** porcentagem de valores convertidos com sucesso  
- **converted:** indica se a coluna foi convertida (baseado no `min_ratio`)

---

### 🔹 Criação automática de features temporais

Para cada coluna convertida em `datetime`, são geradas variáveis derivadas como:

`*_year`, `*_month`, `*_day`, `*_dayofweek`, `*_quarter`, `*_week`,  
`*_is_month_start`, `*_is_month_end`

O prefixo das novas colunas é o **nome original da coluna de data** .

---

### 🧱 Comportamento defensivo

Caso nenhuma coluna seja detectada, a célula não gera erro — apenas loga:

`[dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.`


Assim, o pipeline segue normalmente.

---

### ⚙️ Configuração (`date_cfg`)

| Parâmetro       | Descrição                                                        | Exemplo               |
|-----------------|------------------------------------------------------------------|------------------------|
| `detect_regex`  | Padrão para localizar colunas com nomes de data                 | `"date"`              |
| `explicit_cols` | Lista manual de colunas a converter                             | `["StartDate"]`       |
| `dayfirst`      | Define se o formato é D/M/Y                                     | `True` para 🇧🇷        |
| `utc`           | Define se a conversão deve ser em UTC                           | `False`               |
| `formats`       | Lista de formatos específicos                                   | `["%d/%m/%Y"]`        |
| `min_ratio`     | Fração mínima de parsing bem-sucedido para aceitar a conversão  | `0.8`                 |

---

> 💡 **Resumo:**  
> Esta célula realiza o reconhecimento automático de colunas de data,
converte-as para o formato datetime e gera variáveis derivadas como
ano, mês, dia e semana.
> Caso nenhuma coluna de data seja encontrada, o código é ignorado com segurança,
garantindo a continuidade do pipeline sem erros.
> Os parâmetros em date_cfg permitem ajustar formato, localização e tolerância
de parsing conforme a estrutura de cada dataset.

In [79]:
# --- Tratamento de Datas -----------------------------------------------------
# Usa cfg do arquivo defaults.json (se preferir, mantenha o dict local)
date_cfg = dict(config.get("dates", {})) or {
    "detect_regex": r"(date|data|dt_|_dt$|_date$|_at$|time|timestamp|created|updated)",
    "explicit_cols": [],   # ex.: ["StartDate","EndDate"]
    "dayfirst": False,     # True se D/M/Y (BR)
    "utc": False,
    "formats": [],         # ex.: ["%d/%m/%Y", "%Y-%m-%d"]
    "min_ratio": 0.80,
    "report_path": "date_parse_report.csv"
}

df, parse_report, parsed_cols = ud.parse_dates_with_report_cfg(df, date_cfg)
display(parse_report)

if not parsed_cols:
    logger.info("[dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.")
    # Scanner rápido de suspeitas (em colunas object), útil para alimentar explicit_cols/formats
    obj_cols = df.select_dtypes(include="object").columns.tolist()
    suspects = []
    for c in obj_cols:
        s = pd.to_datetime(df[c], errors="coerce", dayfirst=date_cfg.get("dayfirst", False), utc=date_cfg.get("utc", False))
        ratio = float(s.notna().mean())
        if ratio >= 0.20:  # ajuste conforme seu caso
            suspects.append((c, round(ratio, 3)))
    if suspects:
        print("🔎 Possíveis colunas de data (taxa de parsing com heurística atual):")
        display(pd.DataFrame(suspects, columns=["column","parse_ratio"]).sort_values("parse_ratio", ascending=False))
        print("→ Dica: mova as colunas acima para dates.explicit_cols e/ou informe dates.formats.")
else:
    created = ud.expand_date_features_plus(
        df, parsed_cols,
        features=["year","month","day","dayofweek","quarter","week","is_month_start","is_month_end"],
        prefix_mode="auto"
    )
    logger.info(f"[dates] features criadas: {len(created)}")
    display(df[created].head(10))


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:07,801 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:07,805 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:07,817 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [manifest] step='save_report_df' registrado.
2025-11-04 05:00:07,820 | INFO | [manifest] step='save_report_df' registrado.
[INFO] [report] salvo: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\date_parse_report.csv (0 linhas)
2025-11-04 05:00:07,822 | INFO | [report] salvo: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\date_parse_report.csv (0 linhas)


,column,non_null,parsed,ratio,method,format


2025-11-04 05:00:07,831 | INFO | [dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.


C:\Users\fabio\AppData\Local\Temp\ipykernel_11372\65037927.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  s = pd.to_datetime(df[c], errors="coerce", dayfirst=date_cfg.get("dayfirst", False), utc=date_cfg.get("utc", False))
C:\Users\fabio\AppData\Local\Temp\ipykernel_11372\65037927.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  s = pd.to_datetime(df[c], errors="coerce", dayfirst=date_cfg.get("dayfirst", False), utc=date_cfg.get("utc", False))
C:\Users\fabio\AppData\Local\Temp\ipykernel_11372\65037927.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  s = pd.to_

# 🗓️ Criação da Tabela Calendário (`dim_date`)

Esta etapa gera automaticamente uma **tabela calendário completa** — também chamada de **dimensão de tempo** — a partir de uma coluna de datas existente no dataset principal.  
A tabela é útil para **análises temporais, dashboards e modelos de previsão** que utilizam períodos como referência (ano, mês, trimestre, etc).

---

## 📋 O que acontece aqui

### 🔹 Seleção da coluna de data

Você define manualmente qual coluna será usada como base:

```python
CAL_DATE_COL = "order_date"  # nome da coluna datetime escolhida
CAL_FREQ = "D"               # frequência: "D" (diário), "W" (semanal), "M" (mensal)
```

A coluna deve estar no formato **datetime**.  
Se não estiver, o notebook exibirá um aviso pedindo para executar antes a etapa de **Tratamento de Datas**.

---

### 🔹 Geração da dimensão calendário

A função `build_calendar_from()` constrói uma tabela com todas as datas entre o **mínimo** e o **máximo** encontrados em `CAL_DATE_COL`.

Para cada data, são criadas colunas derivadas, como:

| Coluna          | Descrição                                |
|------------------|-------------------------------------------|
| `date`           | Data base (chave primária)               |
| `year`           | Ano                                      |
| `month`          | Mês numérico                             |
| `day`            | Dia do mês                               |
| `quarter`        | Trimestre (1–4)                          |
| `week`           | Semana do ano (ISO)                      |
| `dow`            | Dia da semana (0 = segunda, 6 = domingo) |
| `is_month_start` | Indica se a data é o primeiro dia do mês |
| `is_month_end`   | Indica se a data é o último dia do mês   |
| `month_name`     | Nome do mês                              |
| `day_name`       | Nome do dia da semana                    |

---

### 🔹 Armazenamento e reuso

A tabela é salva automaticamente no diretório de artefatos do projeto (`reports/artifacts`), usando o utilitário:

```python
CAL_OUT = ud.get_artifacts_dir("calendar") / "dim_date.csv"
ud.save_table(dim_date, CAL_OUT)
```

O formato é determinado pela extensão do arquivo — `.csv` ou `.parquet`.

Além disso, a tabela pode ser registrada no catálogo de DataFrames (`T`) para ser usada em outras partes do pipeline:

```python
T.add("dim_date", dim_date)
```

> 💡 **Resumo:**  
> Esta célula cria uma **dimensão de tempo completa** a partir da coluna de data escolhida.  
> O processo é **automático**, **reproduzível** e **independente do dataset principal**, permitindo:
> - Agregações temporais por ano, mês, semana ou trimestre;  
> - Junções consistentes com outras tabelas de fato;  
> - Integração direta com dashboards (ex.: Power BI).  
> 
> Tudo é salvo e versionado dentro da estrutura padrão do projeto em `reports/artifacts/calendar/`.


In [80]:
# --- validações suaves ---
if CAL_DATE_COL not in df.columns:
    msg = f"⚠️ Coluna '{CAL_DATE_COL}' não encontrada no DataFrame. " \
          f"Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário."
    print(msg); logger.warning(msg)
else:
    if not pd.api.types.is_datetime64_any_dtype(df[CAL_DATE_COL]):
        msg = f"⚠️ Coluna '{CAL_DATE_COL}' existe, mas **não está em formato datetime**. " \
              "Execute a etapa de Tratamento de Datas antes, ou converta manualmente."
        print(msg); logger.warning(msg)
    else:
        # --- construção da dimensão calendário ---
        dim_date = ud.build_calendar_from(df, CAL_DATE_COL, freq=CAL_FREQ)

        # --- visão rápida ---
        display(dim_date.head(12))
        start_date = dim_date["date"].min()
        end_date   = dim_date["date"].max()
        print(f"Período: {start_date.date()} → {end_date.date()}  | Linhas: {len(dim_date)}")

        # --- salvar respeitando a extensão (.csv ou .parquet pelo sufixo)
        ud.save_table(dim_date, CAL_OUT)
        logger.info(f"[calendar] Tabela calendário salva em: {CAL_OUT}")

        # --- opcional: registrar no catálogo de tabelas ---
        try:
            T.add("dim_date", dim_date)
            logger.info("[calendar] 'dim_date' registrada no catálogo T.")
        except NameError:
            pass


⚠️ Coluna 'order_date' não encontrada no DataFrame. Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário.
2025-11-04 05:00:09,237 | WARNING | ⚠️ Coluna 'order_date' não encontrada no DataFrame. Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário.


# 📝 Tratamento de Texto (opcional)

Esta etapa extrai **métricas numéricas e lógicas a partir de colunas textuais**, transformando texto livre em informações quantitativas úteis para **análise exploratória e modelagem**.  

É uma forma leve e controlada de **estruturar dados não numéricos**, sem recorrer a técnicas avançadas de NLP.

---

## 📋 O que acontece aqui

### 🔹 Identificação de colunas textuais

O sistema busca automaticamente colunas com tipo `object` e ignora aquelas listadas na *blacklist*:

```python
text_cols = [c for c in df.columns if df[c].dtype == 'object' and c not in TEXT_CFG["blacklist"]]
```

Essas colunas normalmente contêm informações como:  
descrições, comentários, nomes, categorias textuais ou observações.

---

### 🔹 Limpeza e padronização

Antes de gerar as métricas, os textos passam por uma limpeza leve:
- Remoção de **espaços duplicados** e **trim** nas extremidades.  
- Conversão para **minúsculas**, garantindo consistência em análises de termos.

Essas ações são controladas pelas chaves:
```python
"lower": True,
"strip_collapse_ws": True
```

---

### 🔹 Criação automática de métricas textuais

Para cada coluna textual, são geradas novas colunas numéricas:

| Nova Coluna           | Descrição | Exemplo ("This is great!") |
|------------------------|------------|-----------------------------|
| `<coluna>_len`         | Número total de caracteres no texto | 15 |
| `<coluna>_word_count`  | Número de palavras separadas por espaços | 3 |

Essas novas colunas permitem análises quantitativas sobre padrões de texto e são especialmente úteis em relatórios e modelos de ML que exigem variáveis numéricas.

---

### 🔹 Presença de termos-chave

O sistema também cria colunas booleanas (`True` / `False`) para identificar a **ocorrência de palavras específicas** em cada coluna textual.  

Exemplo:
```python
TEXT_CFG["keywords"] = ["error", "cancel", "premium"]
```

Gera colunas como:
- `<coluna>_has_error`  
- `<coluna>_has_cancel`  
- `<coluna>_has_premium`

Essas variáveis são úteis para análises de sentimento, classificação de textos e detecção de padrões em feedbacks de clientes.

---

### 🔹 Exportação de resumo

Ao final, é gerado um **resumo CSV** com as colunas de texto processadas e suas métricas derivadas.  
O arquivo é salvo automaticamente em:

```
reports/artifacts/text_features/text_features_summary.csv
```

Esse relatório documenta as transformações aplicadas, garantindo **rastreabilidade e transparência** dentro do pipeline de dados.

---

### ⚙️ Configuração (`TEXT_CFG`)

| Parâmetro              | Descrição                                                       | Exemplo                              |
|------------------------|------------------------------------------------------------------|--------------------------------------|
| `lower`                | Converte o texto para minúsculas                                | `True`                               |
| `strip_collapse_ws`    | Remove espaços duplicados e limpa extremidades                  | `True`                               |
| `keywords`             | Lista de termos a detectar no texto                             | `["error", "cancel", "premium"]`     |
| `blacklist`            | Colunas a ignorar durante o processamento                       | `["customerID"]`                     |
| `export_summary`       | Salva relatório com resumo das features geradas                 | `True`                               |

---

### Boas práticas

- Aplique esta etapa apenas em colunas realmente textuais — evite IDs ou códigos.  
- Personalize a lista de **palavras-chave** conforme o contexto do dataset.  
- Caso o dataset não possua colunas de texto, o processo será ignorado com segurança.  
- Utilize o relatório `text_features_summary.csv` para acompanhar colunas derivadas e auditorias.

---

> 💡 **Resumo:**  
> Esta célula transforma campos de texto em **indicadores numéricos e lógicos**, gerando métricas básicas (tamanho, número de palavras e flags de palavras-chave).  
> Tudo é processado automaticamente com **configuração leve e reprodutível**, integrando dados textuais ao pipeline de forma organizada e escalável.


In [81]:
TEXT_CFG = {
    "lower": True,
    "strip_collapse_ws": True,
    "keywords": ["error", "cancel", "premium"],
    "blacklist": ["customerID"],
    "export_summary": True,
}

if config.get("text_features", True):
    summary_path = ud.get_artifacts_dir("text_features")
    df, text_summary = ud.extract_text_features(
        df,
        lower=TEXT_CFG["lower"],
        strip_collapse_ws=TEXT_CFG["strip_collapse_ws"],
        keywords=TEXT_CFG["keywords"],
        blacklist=TEXT_CFG["blacklist"],
        export_summary=TEXT_CFG["export_summary"],
        summary_dir=summary_path
    )
    display(text_summary.head())


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:09,952 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\text_features
2025-11-04 05:00:09,955 | INFO | [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\text_features
[INFO] [text] resumo salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\text_features\text_features_summary.csv (16 colunas)
2025-11-04 05:00:10,369 | INFO | [text] resumo salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\text_features\text_features_summary.csv (16 colunas)


C:\Users\fabio\Projetos DEV\data projects\data-project-template\utils\utils_data.py:1629: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out[kw_flag] = s.str.contains(rf"\b{kw}\b", regex=True, na=False)


,column,non_null,avg_len,avg_words,kw_error_count,kw_cancel_count,kw_premium_count
0,Churn,7043,2.27,1.00,0,0,0
1,Contract,7043,11.30,1.45,0,0,0
2,Dependents,7043,2.30,1.00,0,0,0
3,DeviceProtection,7043,6.03,1.43,0,0,0
4,InternetService,7043,6.30,1.44,0,0,0


# 🔤 Codificação de Categóricas & 🔢 Escalonamento Numérico (opcionais)

Esta etapa transforma variáveis **categóricas** em representações numéricas e **padroniza a escala** de variáveis **numéricas** quando necessário.  
É útil para alimentar modelos de ML que **não aceitam strings** (ex.: regressões, SVMs, redes neurais) e/ou **são sensíveis à escala** (KNN, SVM com kernel RBF, PCA, etc.).

---

## 🧭 Visão Geral

- **Codificação (Encoding)**
  - **One-hot**: cria uma coluna por categoria (0/1). É a opção **mais segura** para baseline; evita ordenar categorias artificialmente.
  - **Ordinal**: mapeia categorias para números inteiros. Mais **compacto**, mas induz **ordem artificial**; use com cuidado quando **não** há ordem natural.

- **Escalonamento (Scaling)**
  - **Standard**: `z = (x - média) / desvio`; centra em 0 com variância ≈ 1. Bom para dados **aprox. gaussianos**.
  - **MinMax**: escala para **[0, 1]**; útil quando limites mínimos e máximos são relevantes (redes neurais, normalizações simples).

---

## ⚙️ Configuração usada no notebook

As opções abaixo são lidas do `config` e repassadas para a função utilitária do template (`apply_encoding_and_scaling`, quando habilitada).

```python
ENCODE_CFG = {
    "enabled":              config.get("encode_categoricals", True),
    "type":                 config.get("encoding_type", "onehot"),  # "onehot" | "ordinal"
    "exclude_cols":         ["Churn", "customerID"],                # NÃO codificar alvo/ids
    "high_card_threshold":  50,                                     # ignora colunas com cardinalidade muito alta
}

SCALE_CFG = {
    "enabled":              config.get("scale_numeric", False),
    "method":               config.get("scaler", "standard"),       # "standard" | "minmax"
    "exclude_cols":         ["Churn"],                              # NÃO escalar o alvo
    "only_continuous":      True,                                   # evita escalar dummies e inteiros-discretos
}

df, encoding_meta, scaling_meta = apply_encoding_and_scaling(
    df, encode_cfg=ENCODE_CFG, scale_cfg=SCALE_CFG
)
```

> 💡 **Dica:** deixe `scale_numeric = false` no início do projeto (exploração/entendimento).  
> Ative a escala somente quando partir para **modelagem** e **validação**.

---

## 🧩 O que a função faz (alto nível)

1. **Seleciona colunas categóricas** (`object`/`category`) **excluindo** as listadas em `exclude_cols` e as com **cardinalidade > high_card_threshold** (proteção contra explosão de dummies).
2. **Codifica** conforme `type`:
   - `onehot` → usa `pd.get_dummies(..., dtype=float)` e concatena ao DataFrame (removendo as originais).
   - `ordinal` → usa `sklearn.preprocessing.OrdinalEncoder` (com `handle_unknown='use_encoded_value'` e `unknown_value=-1`).
3. **Escala colunas numéricas** se `SCALE_CFG["enabled"]`:
   - seleciona **apenas** colunas numéricas **contínuas** quando `only_continuous=True` (tipicamente `float`/amplitude alta).
   - aplica `StandardScaler` **ou** `MinMaxScaler` do scikit-learn.
4. Retorna:
   - `df` (atualizado),
   - `encoding_meta` (categorias vistas, tipo de codificação, colunas excluídas, descartes por cardinalidade),
   - `scaling_meta` (tipo de escala, colunas escaladas, parâmetros do scaler).

---

## 📦 Saídas e Metadados

**`encoding_meta`** inclui:
- `encoding`: `"onehot"` **ou** `"ordinal"`  
- `excluded`: lista de colunas ignoradas (ex.: `["Churn","customerID"]`)  
- `high_card_excluded`: colunas **não encodadas** por alta cardinalidade  
- `categorical_columns`: colunas categóricas processadas  
- (para ordinal) `categories_`: categorias aprendidas por coluna

**`scaling_meta`** inclui:
- `scaler`: `"standard"` **ou** `"minmax"`  
- `scaled_columns`: lista das colunas escaladas  
- `means_`/`scales_` (Standard) **ou** `min_`/`range_` (MinMax) — úteis para **reprodução** no deploy

---

## 🧪 Exemplo de comportamento esperado

Dataset: `gender` (`Male`/`Female`), `InternetService` (DSL/Fiber/No), `MonthlyCharges` (float), `tenure` (int), `Churn` (alvo).

Com `onehot` + `standard` e exclusões padrão, o resultado será:
- Novas colunas como `gender_Female`, `gender_Male`, `InternetService_DSL`, etc.
- `MonthlyCharges` escalado (média≈0, desvio≈1); `tenure` **pode** ser ignorado do scaling se `only_continuous=True` e considerado discreto.

---

## 🚧 Armadilhas comuns e proteções do template

- **Explosão de dummies**: colunas com **muitas categorias** são ignoradas (registradas em `high_card_excluded`).  
- **Vazamento do alvo**: garanta que `exclude_cols` contenha a **target** (ex.: `"Churn"`).  
- **IDs**: não codificar/escalar IDs (`customerID`).  
- **Mixed types**: colunas mal tipadas (número como string) devem ser padronizadas antes (ver **Qualidade & Tipagem**).

---

## ✅ Status de execução e logs

Mensagens típicas no log:

```
[encode] type=onehot | cols=['gender', 'InternetService'] | high_card_excluded=[]
[scale] method=standard | scaled_cols=['MonthlyCharges']
```

Se o log mostrar `cols=[]`, significa que **não há colunas categóricas** elegíveis (ou foram excluídas por configuração/cardinalidade). Isso é **normal** em alguns datasets.

---

## 🧠 Boas práticas

- Comece com **one-hot** e **sem escala** para uma baseline interpretável.
- Use **ordinal** só quando houver **ordem natural** nas categorias.
- Só escale **depois** do split treino/validação para evitar vazamento.
- Salve `encoding_meta` e `scaling_meta` se for levar o modelo para produção.

---

> 💡 **Resumo:**  
> Esta célula converte dados categóricos e numéricos em formatos ideais para modelagem, controlando exclusões, cardinalidade e escala — garantindo **robustez**, **consistência** e **clareza** no tratamento.


In [82]:
# --- 🔤 Tratamento de Texto --------------------------------------------------
TEXT_CFG = {
    "lower": True,
    "strip_collapse_ws": True,
    "keywords": ["error", "cancel", "premium"],
    "blacklist": ["customerID"],
    "export_summary": True,
}

if config.get("text_features", True):
    summary_dir = ud.get_artifacts_dir("text_features")

    df, text_summary = ud.extract_text_features(
        df,
        lower=TEXT_CFG["lower"],
        strip_collapse_ws=TEXT_CFG["strip_collapse_ws"],
        keywords=TEXT_CFG["keywords"],
        blacklist=TEXT_CFG["blacklist"],
        export_summary=TEXT_CFG["export_summary"],
        summary_dir=summary_dir
    )

    display(text_summary.head(10))


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:15,310 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\text_features
2025-11-04 05:00:15,312 | INFO | [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\text_features
[INFO] [text] resumo salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\text_features\text_features_summary.csv (16 colunas)
2025-11-04 05:00:15,767 | INFO | [text] resumo salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\text_features\text_features_summary.csv (16 colunas)


,column,non_null,avg_len,avg_words,kw_error_count,kw_cancel_count,kw_premium_count
0,Churn,7043,2.27,1.00,0,0,0
1,Contract,7043,11.30,1.45,0,0,0
2,Dependents,7043,2.30,1.00,0,0,0
3,DeviceProtection,7043,6.03,1.43,0,0,0
4,InternetService,7043,6.30,1.44,0,0,0
5,MultipleLines,7043,3.78,1.19,0,0,0
6,OnlineBackup,7043,6.03,1.43,0,0,0
7,OnlineSecurity,7043,5.97,1.43,0,0,0
8,PaperlessBilling,7043,2.59,1.00,0,0,0
9,Partner,7043,2.48,1.00,0,0,0


# 🎯 Criação e Validação da Variável-Alvo (`target`)

Esta etapa garante que o dataset contenha uma **variável-alvo binária** adequada para **modelos supervisionados de Machine Learning** — como classificação de churn, fraude, aprovação, entre outros.  

O processo é flexível e segue as instruções definidas no arquivo `config/defaults.json`, permitindo que a criação do *target* seja **reproduzível e automática** em qualquer dataset.

---

## ⚙️ Configuração do Target

O comportamento é controlado pela seção `"target"` do arquivo de configuração.  
Exemplo típico (como no dataset *Telco Customer Churn*):

```json
"target": {
  "name": "Churn",
  "source": "Churn",
  "positive": "Yes",
  "negative": "No"
}
```

### Parâmetros
| Chave | Descrição | Exemplo |
|--------|------------|----------|
| `name` | Nome final da coluna de destino | `"Churn"` |
| `source` | Coluna de origem (se o nome for diferente) | `"Churn"` |
| `positive` | Valor que representa a classe positiva | `"Yes"` |
| `negative` | Valor que representa a classe negativa | `"No"` |

> 💡 **Dica:** se o dataset já tiver a coluna-alvo, a função apenas validará e manterá como está — sem sobrescrever.

---

## 🧩 O que o código faz

```python
df, target_name, class_map, tgt_report = ud.ensure_target_from_config(df, config, verbose=True)
display(tgt_report)
globals()["class_map"] = class_map
```

### Etapas internas:
1. **Verifica se a coluna-alvo já existe**
   - Se existir (`Churn`), exibe mensagem:  
     ```
     [target] Coluna 'Churn' já existe — nenhuma ação necessária.
     ```
   - Caso contrário, cria automaticamente a nova coluna de acordo com o `config`.

2. **Aplica o mapeamento definido**
   - Exemplo: `Yes → 1`, `No → 0`.

3. **Retorna informações estruturadas**
   - `target_name`: nome da coluna alvo  
   - `class_map`: dicionário de mapeamento de classes  
   - `tgt_report`: pequeno relatório em formato DataFrame, exibindo status da operação

---

## 📊 Mini-relatório de validação

Após a execução, é exibido algo como:

| target | status     | source | positive | negative |
|---------|-------------|---------|-----------|-----------|
| Churn   | já existe  | Churn   | Yes       | No        |

Esse relatório resume como o target foi identificado ou criado, garantindo rastreabilidade.

---

## 🔍 Verificação de consistência

Após criar (ou validar) a coluna-alvo, o notebook checa o conteúdo e tipo:

```python
print(df["Churn"].unique())
print(df["Churn"].value_counts())
```

Resultados esperados:

| Caso | Exemplo de saída | Interpretação |
|------|------------------|---------------|
| Target binário correto | `[0, 1]` | ✅ Pronto para ML |
| Target textual | `['Yes', 'No']` | 🔄 Converter com `.map({'Yes':1,'No':0})` |
| Target booleano | `[True, False]` | 🔄 Converter com `.astype(int)` |

---

## 🧠 Boas práticas

- Defina claramente qual valor representa a **classe positiva** (ex.: `"Yes"`, `"Exited"`, `"Churned"`).  
- Evite múltiplas colunas com o mesmo significado — centralize em uma só (`target` ou `Churn`).  
- Para modelos multiclasse, amplie o mapeamento conforme necessário (ex.: `"Gold"`, `"Silver"`, `"Bronze"`).  
- Sempre valide a distribuição de classes com `value_counts()` antes da modelagem.

---

## 🚀 Integração com notebooks seguintes

A variável `class_map` é registrada globalmente para que o **Notebook N2** (modelagem) possa reutilizá-la:

```python
globals()["class_map"] = class_map
```

Assim, o mapeamento de classes permanece disponível para:
- Conversões de rótulos (`1 → 'Yes'`, `0 → 'No'`);
- Exibição de resultados e métricas em termos legíveis.

---

> 💡 **Resumo:**  
> Esta célula garante que o dataset contenha uma variável-alvo binária padronizada, compatível com os modelos de ML e definida de forma reprodutível via arquivo de configuração.  
> Se a coluna já existir, é validada e mantida; se não, é criada conforme o mapeamento indicado no `config`.


In [83]:
# Cria o target a partir da config (ou defaults)
df, target_name, class_map, tgt_report = ud.ensure_target_from_config(df, config, verbose=True)

# Opcional: ver um mini-relatório
display(tgt_report)

# Garantir que o N2 enxergue o mapeamento, se você quiser usar class_map lá:
globals()["class_map"] = class_map

print('Verificando a consistência do tipo e dos valores:')
print(df["Churn"].unique())
print(df["Churn"].value_counts())

[target] Coluna 'Churn' já existe — nenhuma ação necessária.


,target,status,source,positive,negative
0,Churn,já existe,Churn,Yes,No


Verificando a consistência do tipo e dos valores:
['no' 'yes']
Churn
no     5174
yes    1869
Name: count, dtype: int64


# 📦 Exportação de Artefatos (N1 → N2)

Esta etapa consolida e exporta os **artefatos finais do notebook N1** — garantindo que os dados tratados, os metadados e o manifesto de execução estejam disponíveis para o **N2 (Modelagem)**.  

É o fechamento da fase de preparação dos dados, onde tudo que foi limpo, transformado e enriquecido é salvo em **arquivos persistentes e reprodutíveis**.

---

## 🧭 Visão Geral do Processo

1. **Exporta os dados tratados** (`interim` e `processed`);
2. **Gera um arquivo de metadados (`meta.json`)** com a estrutura do dataset;
3. **Cria um manifesto (`manifest.json`)** com o resumo técnico da execução — memória usada, flags criadas, caminhos de saída, etc.

Tudo é salvo em caminhos padronizados e definidos dinamicamente a partir do `config`.

---

## ⚙️ Configurações e Caminhos Padrão

Os caminhos de saída podem ser definidos no `config/defaults.json`, ou usam valores padrão se ausentes:

| Chave | Caminho padrão | Descrição |
|--------|----------------|-----------|
| `data_interim_file` | `data/interim/interim.parquet` | Dados intermediários tratados |
| `data_processed_file` | `data/processed/processed.parquet` | Dados finais prontos para modelagem |
| `meta_file` | `artifacts/metadata/dataset_meta.json` | Metadados do dataset |
| `random_seed` | `42` (padrão) | Semente para reprodutibilidade |

A função `get_artifacts_dir("export")` cria automaticamente o diretório padrão de exportação:

```
reports/artifacts/export/
```

---

## 📋 Etapas principais do código

### 1️⃣ Exportação dos DataFrames

```python
OUTPUT_PROCESSED = Path(config.get("data_processed_file", "data/processed/processed.parquet"))
OUTPUT_INTERIM   = Path(config.get("data_interim_file",   "data/interim/interim.parquet"))

if config.get("export_interim", True):
    _save_df(df, OUTPUT_INTERIM)

if config.get("export_processed", True):
    _save_df(df, OUTPUT_PROCESSED)
```

A função `_save_df()` detecta automaticamente o formato pelo sufixo do arquivo:

| Extensão | Função utilizada | Compressão |
|-----------|------------------|-------------|
| `.parquet` | `to_parquet()` | Compactado (`snappy`) |
| `.csv` | `to_csv()` | UTF-8 |
| `.xlsx` | `to_excel()` | Excel |

---

### 2️⃣ Geração dos Metadados (`meta.json`)

Cria um registro detalhado da estrutura do dataset exportado.

```python
meta = {
  "dataset_name": config.get("dataset_name", "Dataset"),
  "version": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
  "target": target_col,
  "class_map": class_map,
  "columns": {
    "numeric": numeric_cols,
    "categorical": categorical_cols,
    "boolean": boolean_cols,
    "ignored": ignored_cols,
    "all": all_cols
  }
}
META_FILE.write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8")
```

Esse arquivo é essencial para o N2, pois indica:
- quais colunas são numéricas, categóricas e booleanas;
- qual é o `target`;
- quais colunas foram ignoradas;
- o mapeamento de classes (`class_map`, se existir);
- a versão e data de exportação.

> 💡 **Dica:**  
> você pode adicionar `meta["dtypes"] = {c: str(t) for c, t in df.dtypes.items()}`  
> para registrar os tipos exatos de cada coluna no arquivo de metadados.

---

### 3️⃣ Registro do Manifesto de Execução (`manifest.json`)

O manifesto é um **resumo técnico** da sessão atual — útil para auditoria e rastreabilidade.

```python
manifest = {
  "created_at": datetime.now().isoformat(timespec="seconds"),
  "random_seed": RANDOM_SEED,
  "config": config,
  "memory_mb": float(df.memory_usage(deep=True).sum() / (1024**2)),
  "outlier_flags": [c for c in df.columns if c.endswith("_is_outlier")],
  "imputed_flags": [c for c in df.columns if c.startswith("was_imputed_")],
  "shape": tuple(df.shape),
  "exported": {
    "interim": str(OUTPUT_INTERIM),
    "processed": str(OUTPUT_PROCESSED),
    "meta_file": str(META_FILE)
  }
}

(ARTIFACTS_DIR / "manifest.json").write_text(json.dumps(manifest, indent=2, ensure_ascii=False))
```

Campos mais importantes:
| Campo | Descrição |
|--------|------------|
| `created_at` | Data e hora da execução |
| `random_seed` | Semente aleatória utilizada |
| `memory_mb` | Tamanho em memória do DataFrame |
| `outlier_flags` / `imputed_flags` | Colunas criadas por outliers ou imputações |
| `shape` | Dimensão final do dataset |
| `exported` | Caminhos de saída dos artefatos gerados |

---

### 4️⃣ Log de conclusão

Ao final da célula, é exibido no notebook:

```
[INFO] [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\export
Arquivos gerados:
- INTERIM:   data\interim\interim.parquet
- PROCESSED: data\processed\processed.parquet
- META:      artifacts\metadata\dataset_meta.json
```

---

## 🧠 Boas práticas

- **Versão dos dados**: use o campo `"version"` do meta para marcar reprocessamentos.
- **Controle de reprodutibilidade**: defina `random_seed` no config.
- **Organização**: mantenha sempre `interim`, `processed` e `meta` sob versionamento no Git.
- **Auditoria**: o `manifest.json` é a “assinatura digital” da execução do N1.
- **Portabilidade**: todo o caminho é relativo à raiz do projeto, garantindo compatibilidade com outros ambientes.

---

> 💡 **Resumo:**  
> Esta célula encerra o **notebook N1**, gerando todos os artefatos necessários para as etapas seguintes (EDA, modelagem e visualização).  
> Ela garante **reprodutibilidade**, **rastreamento** e **organização** dos resultados, consolidando as saídas do pipeline em formatos padronizados e facilmente reutilizáveis.


In [84]:
# =============================================================================
# 📦 Exportação de Artefatos (N1 → N2)
# =============================================================================
from datetime import datetime
from pathlib import Path
import json
import numpy as np
import pandas as pd

# 0) Raiz do projeto
PROJECT_ROOT = ud.ensure_project_root()
logger.info(f"[path] PROJECT_ROOT -> {PROJECT_ROOT}")

# 1) Caminhos a partir da config (com defaults) — relativos à raiz
OUTPUT_PROCESSED = PROJECT_ROOT / Path(config.get("data_processed_file", "data/processed/processed.parquet"))
OUTPUT_INTERIM   = PROJECT_ROOT / Path(config.get("data_interim_file",   "data/interim/interim.parquet"))
META_FILE        = PROJECT_ROOT / Path(config.get("meta_file",           "artifacts/metadata/dataset_meta.json"))

# 2) Garante diretórios
OUTPUT_PROCESSED.parent.mkdir(parents=True, exist_ok=True)
OUTPUT_INTERIM.parent.mkdir(parents=True, exist_ok=True)
META_FILE.parent.mkdir(parents=True, exist_ok=True)

# 👉 uso do helper para resolver o diretório de artefatos (sempre na raiz)
ARTIFACTS_DIR = ud.get_artifacts_dir("export")

# 3) Salva tabelas respeitando a extensão (.csv / .parquet / .xlsx)
def _save_df(df_, path_: Path):
    ext = path_.suffix.lower()
    if ext == ".parquet":
        df_.to_parquet(path_, index=False)
    elif ext == ".csv":
        df_.to_csv(path_, index=False, encoding="utf-8")
    elif ext == ".xlsx":
        df_.to_excel(path_, index=False)
    else:
        raise ValueError(f"Extensão não suportada: {ext}")

if config.get("export_interim", True):
    _save_df(df, OUTPUT_INTERIM)

if config.get("export_processed", True):
    _save_df(df, OUTPUT_PROCESSED)

# 4) Monta METADADOS para o N2 (alinhado ao config.target.name, com fallback)
target_cfg  = config.get("target", {}) or {}
target_col  = target_cfg.get("name", config.get("target_column", "target"))

all_cols = df.columns.tolist()
ignored_cols = config.get("ignored_columns", [])
candidate_features = [c for c in all_cols if c not in ignored_cols and c != target_col]

numeric_cols     = df[candidate_features].select_dtypes(include=[np.number]).columns.tolist()
boolean_cols     = df[candidate_features].select_dtypes(include=["bool", "boolean"]).columns.tolist()
categorical_cols = [c for c in candidate_features if c not in numeric_cols]

class_map = globals().get("class_map", None)

meta = {
    "dataset_name": config.get("dataset_name", "Dataset"),
    "version":      datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "target":       target_col,
    "class_map":    class_map,
    "columns": {
        "numeric":     numeric_cols,
        "categorical": categorical_cols,
        "boolean":     boolean_cols,
        "ignored":     ignored_cols,
        "all":         all_cols
    }
}

# enriquecer com esquema e nº de linhas
meta["dtypes"] = {c: str(t) for c, t in df.dtypes.items()}
meta["rows"]   = int(len(df))

# 5) Salva META
META_FILE.write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8")

# 🔍 Verificação e resumo final
_mpath = Path(META_FILE)
_mjson = json.loads(_mpath.read_text(encoding="utf-8"))
print("\n" + "="*80)
print("📦  EXPORTAÇÃO DE ARTEFATOS — RESUMO FINAL")
print("="*80)
print(f"🗂️  Dataset:        {meta['dataset_name']}")
print(f"🎯  Target:         {_mjson.get('target')}")
print(f"🧮  Linhas:         {_mjson.get('rows'):,}")
print(f"🔢  Numéricas:      {len(_mjson['columns']['numeric'])} colunas")
print(f"🔠  Categóricas:    {len(_mjson['columns']['categorical'])} colunas")
print(f"🔘  Booleanas:      {len(_mjson['columns']['boolean'])} colunas")
print(f"🧾  Ignoradas:      {len(_mjson['columns']['ignored'])} colunas")
print(f"🧩  Todas:          {len(_mjson['columns']['all'])} colunas totais")
print("="*80 + "\n")


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:17,596 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:17,599 | INFO | [path] PROJECT_ROOT -> C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:17,601 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\export
2025-11-04 05:00:17,603 | INFO | [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\export

📦  EXPORTAÇÃO DE ARTEFATOS — RESUMO FINAL
🗂️  Dataset:        Dataset
🎯  Target:         Churn
🧮  Linhas:         7,043
🔢  Numéricas:      62 colunas
🔠  Categóricas:    65 colunas
🔘  Booleanas:      49 colunas
🧾  Ignorada

# 🧭 Resumo final de sanidade

In [85]:
# =========================================================
# ✅ Resumo final de sanidade do N1
# =========================================================
from pathlib import Path

ARTIFACTS_DIR_EXPORT = ud.get_artifacts_dir("export")  # garante caminho
manifest_path = ARTIFACTS_DIR_EXPORT / "manifest.json"

print("\n" + "="*80)
print("✅  N1 CONCLUÍDO — RESUMO DE SANIDADE")
print("="*80)

print(f"📐 Shape final do df:   {df.shape}")
print(f"🎯 Target:              {meta.get('target', '(desconhecido)')}")
print(f"🧮 Linhas registradas:  {meta.get('rows', '(n/a)')}")
print(f"🧾 Colunas (totais):    {len(meta.get('columns', {}).get('all', []))}")
print(f"   ├─ Numéricas:        {len(meta.get('columns', {}).get('numeric', []))}")
print(f"   ├─ Categóricas:      {len(meta.get('columns', {}).get('categorical', []))}")
print(f"   └─ Booleanas:        {len(meta.get('columns', {}).get('boolean', []))}")

print("\n📦 Artefatos gerados:")
print(f"   ├─ INTERIM:          {Path(OUTPUT_INTERIM).exists()} | {OUTPUT_INTERIM}")
print(f"   ├─ PROCESSED:        {Path(OUTPUT_PROCESSED).exists()} | {OUTPUT_PROCESSED}")
print(f"   ├─ META:             {Path(META_FILE).exists()} | {META_FILE}")
print(f"   └─ MANIFEST:         {manifest_path.exists()} | {manifest_path}")

try:
    mem_mb = float(df.memory_usage(deep=True).sum() / (1024**2))
    print(f"\n🧠 Memória do DF:       {mem_mb:,.2f} MB")
except Exception:
    pass

print("="*80 + "\n")


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
2025-11-04 05:00:18,962 | INFO | PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\export
2025-11-04 05:00:18,964 | INFO | [path] artifacts_dir -> C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\artifacts\export

✅  N1 CONCLUÍDO — RESUMO DE SANIDADE
📐 Shape final do df:   (7043, 128)
🎯 Target:              Churn
🧮 Linhas registradas:  7043
🧾 Colunas (totais):    128
   ├─ Numéricas:        62
   ├─ Categóricas:      65
   └─ Booleanas:        49

📦 Artefatos gerados:
   ├─ INTERIM:          True | C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\interim\interim.parquet
   ├─ PROCESSED:        True | C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\processed\processed.parquet
   ├─ META:             Tr

## ✅ Checkpoint

- **Revisar** as colunas derivadas e decisões (imputação, outliers, codificação).  
- **Documentar** no README as escolhas de negócio e justificativas.  

## 📎 Anotações

Esta seção pode ser usada como bloco livre para observações específicas do projeto.
